In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
Pdata= pd.read_csv('AuctionData.csv')
PositionCTR=pd.read_csv("CTR10.csv")

In [3]:
path="Keyword6"

In [4]:
GFPK1=pd.read_csv(path+"\BidMatrixGFP.csv")
GSPK1=pd.read_csv(path+"\BidMatrixGSP.csv")
QualityScore=pd.read_csv(path+"\QualityScore.csv")
ClickRate=pd.read_csv(path+"\Clickrate.csv")
BidData=pd.read_csv(path+"\BidArray.csv")

In [5]:
class Slot:
    CTR=-1
    Owner="NoName"
    impression=0
    click=0
    Cost=0

    def __init__(self, Ctr,imp,c):
        self.CTR=Ctr
        self.impression=imp
        self.click=c

In [6]:
class Bid:
    bid=-1
    bidder="NoName"
    quality=1
    
    def __init__(self, Bidder,Bid,qua):
        self.bid=Bid
        self.bidder=Bidder
        self.quality=qua

In [154]:
class Auction:
    KeywordID=-1
    bids=[]   
    def __init__(self,K_ID,bid):
        self.bids=[]
        self.KeywordID = K_ID
        for i in range(len(bid)):
            self.bids.append(bid[i])
# VCG Mechanism
    def VCG(self,slots):
        length = len(slots) 
        for i in range(len(slots)):
            sum=0          
            for j in range(i+1,len(slots)+1):
                if j!=(len(slots)):
                    sum+=self.bids[j].bid*(slots[j-1].CTR - slots[j].CTR)
                else:
                    sum+=self.bids[j].bid*(slots[j-1].CTR - 0)                  
            slots[i].Cost=sum*(1/slots[i].CTR)
        for i in range(0, length):
            slots[i].Owner = self.bids[i].bidder      
# GSP Mechansim  
    def GSP(self,slots):
        length = len(slots)        
        for i in range(len(self.bids)):
            for j in range(i+1,len(self.bids)):
                if (self.bids[i].bid*self.bids[i].quality)<(self.bids[j].bid*self.bids[j].quality):
                    temp=self.bids[i]
                    self.bids[i]=self.bids[j]
                    self.bids[j]=temp
        for i in range(length):
            slots[i].Cost =(self.bids[i+1].bid*self.bids[i+1].quality)/self.bids[i].quality
        for i in range(0, length):
            slots[i].Owner = self.bids[i].bidder
# GSP Mechansim--NO Quality score           
    def GSPnq(self,slots):
        length=len(slots)      
        for i in range(length):
            slots[i].Cost=self.bids[i+1].bid  
        for i in range(length):
            slots[i].Owner = self.bids[i].bidder    
# GFP Mechansim           
    def GFP(self,slots):
        length=len(slots)
        
        for i in range(length):
            slots[i].Cost=self.bids[i].bid
        
        for i in range(length):
            slots[i].Owner = self.bids[i].bidder

In [7]:
def SortBid(Bid):
    for i in range(len(Bid)):
        for j in range(i+1,len(Bid)):
            if Bid[i][1]<Bid[j][1]:
                Bt=Bid[j][1]
                At=Bid[j][0]
                Bid[j][1]=Bid[i][1]
                Bid[j][0]=Bid[i][0]
                Bid[i][1]=Bt
                Bid[i][0]=At
    return Bid

In [8]:
def FindQScore(BidderName,Quality):
    for i in range(len(Quality)):
        if Quality['BidderID'][i]==BidderName:
            return Quality['Quality'][i]

In [9]:
def TrueValue(Bidder,List):
    for i in range(len(List)):
        if List['BidderID'][i]==Bidder:
            return List['Price'][i]

In [10]:
def CalculateClick(position,bidder,ClickRate,pCTR):
    for i in range(len(ClickRate)):
        if bidder==ClickRate['Bidder'][i]:
            CR=ClickRate['ClickRate'][i]
            break
    return pCTR['Impression'][position]*CR

# Put data into Class

In [11]:
BidO=np.empty((20,2),dtype='object')
for i in range(20):
    BidO[i][0]=GFPK1['Bidder'][i]
    BidO[i][1]=GFPK1['B1'][i]
BidO=SortBid(BidO)

bidList=[]
for i in range(20):
    q=FindQScore(BidO[i][0],QualityScore)
    b=Bid(BidO[i][0],BidO[i][1],q)
    bidList.append(b)
del BidO
slotList=[]
for i in range(10):
    s=Slot(PositionCTR['ClickThroughRate'][i],PositionCTR['Impression'][i],PositionCTR['Click'][i])
    slotList.append(s)

In [160]:
BidO=np.empty((20,2),dtype='object')
for i in range(20):
    BidO[i][0]=GSPK1['BidderID'][i]
    BidO[i][1]=GSPK1['Bid'][i]
BidO=SortBid(BidO)
GSPBid=[]
for i in range(20):
    q=FindQScore(BidO[i][0],QualityScore)
    b=Bid(BidO[i][0],BidO[i][1],q)
    GSPBid.append(b)
del BidO

In [161]:
BidO=np.empty((20,2),dtype='object')
for i in range(20):
    BidO[i][0]=BidData['BidderID'][i]
    BidO[i][1]=BidData['Price'][i]
BidO=SortBid(BidO)
VCGBid=[]
for i in range(20):
    q=FindQScore(BidO[i][0],QualityScore)
    b=Bid(BidO[i][0],BidO[i][1],q)
    VCGBid.append(b)
del BidO

Slots' information

In [13]:
for i,slot in enumerate(slotList):
    print("Slot %2.0f:    CTR: %.5f    impression: %8.0f    Click: %6.0f"%(i+1,slot.CTR,slot.impression,slot.click))

Slot  1:    CTR: 0.01855    impression: 47111917    Click: 874080
Slot  2:    CTR: 0.01022    impression: 45412275    Click: 464205
Slot  3:    CTR: 0.00727    impression: 41105765    Click: 299045
Slot  4:    CTR: 0.00488    impression: 37681588    Click: 183735
Slot  5:    CTR: 0.00427    impression: 32541800    Click: 139014
Slot  6:    CTR: 0.00396    impression: 17732749    Click:  70195
Slot  7:    CTR: 0.00322    impression: 16145931    Click:  52066
Slot  8:    CTR: 0.00284    impression: 15237973    Click:  43231
Slot  9:    CTR: 0.00271    impression: 13933974    Click:  37825
Slot 10:    CTR: 0.00270    impression: 13313311    Click:  35940


Bidders' information of GFP

In [163]:
for i in range(len(bidList)):
    print("Bidder ID:%s, Bid: %4.0f, Quality: %.4f"% (bidList[i].bidder,bidList[i].bid,bidList[i].quality))

Bidder ID:ae30807c-5316-42ee-a031-589fdb08e4ba, Bid: 22440, Quality: 1.1974
Bidder ID:52852966-92dd-43ac-8435-01b9e6021c7c, Bid: 8000, Quality: 0.0000
Bidder ID:6524d297-40d9-4a96-90c0-0d5aa0c61da6, Bid: 3912, Quality: 2.4105
Bidder ID:54bff083-a345-436d-99f4-3e70f8969567, Bid: 3000, Quality: 0.5769
Bidder ID:a2860a55-f8a1-4d62-8edc-49fb88661adc, Bid: 1600, Quality: 1.5453
Bidder ID:ebd4f3cf-2d58-41d9-a848-e3db688ba491, Bid: 1600, Quality: 1.1903
Bidder ID:b5f02143-b63e-4062-bb82-0624909e55ac, Bid: 1200, Quality: 1.1144
Bidder ID:9eb5a22f-1870-42b1-aaae-2faa5235ae34, Bid:  800, Quality: 4.4889
Bidder ID:223f15fb-45b3-4a53-bdcf-c173a6e8c38a, Bid:  680, Quality: 0.0000
Bidder ID:10fa3930-fa02-4c4a-8fd6-4bc5d831c171, Bid:  480, Quality: 0.0000
Bidder ID:a980abdb-d307-4aa8-99f1-03e268964e0b, Bid:  480, Quality: 1.1905
Bidder ID:bcbad0a2-c502-44ba-9e8a-16ade955ece5, Bid:  400, Quality: 0.1961
Bidder ID:cea14acb-7635-41b6-a3fd-d05468e33042, Bid:  400, Quality: 0.0000
Bidder ID:7484012b-fa40-

Bidders' information of GSP

In [164]:
for i in range(len(bidList)):
    print("Bidder ID:%s, Bid: %4.0f, Quality: %.4f"% (GSPBid[i].bidder,GSPBid[i].bid,GSPBid[i].quality))

Bidder ID:ae30807c-5316-42ee-a031-589fdb08e4ba, Bid: 28050, Quality: 1.1974
Bidder ID:52852966-92dd-43ac-8435-01b9e6021c7c, Bid: 6025, Quality: 0.0000
Bidder ID:6524d297-40d9-4a96-90c0-0d5aa0c61da6, Bid: 2784, Quality: 2.4105
Bidder ID:54bff083-a345-436d-99f4-3e70f8969567, Bid: 1932, Quality: 0.5769
Bidder ID:a2860a55-f8a1-4d62-8edc-49fb88661adc, Bid: 1037, Quality: 1.5453
Bidder ID:ebd4f3cf-2d58-41d9-a848-e3db688ba491, Bid:  901, Quality: 1.1903
Bidder ID:b5f02143-b63e-4062-bb82-0624909e55ac, Bid:  814, Quality: 1.1144
Bidder ID:9eb5a22f-1870-42b1-aaae-2faa5235ae34, Bid:  658, Quality: 4.4889
Bidder ID:223f15fb-45b3-4a53-bdcf-c173a6e8c38a, Bid:  611, Quality: 0.0000
Bidder ID:10fa3930-fa02-4c4a-8fd6-4bc5d831c171, Bid:  600, Quality: 0.0000
Bidder ID:a980abdb-d307-4aa8-99f1-03e268964e0b, Bid:  600, Quality: 1.1905
Bidder ID:bcbad0a2-c502-44ba-9e8a-16ade955ece5, Bid:  500, Quality: 0.1961
Bidder ID:cea14acb-7635-41b6-a3fd-d05468e33042, Bid:  500, Quality: 0.0000
Bidder ID:7484012b-fa40-

# Start GFP Auction

In [165]:
KeyWord="79021a2e2c836c1a"

In [59]:
auctionGFP = Auction(KeyWord,bidList)

In [60]:
auctionGFP.GFP(slotList)

In [26]:
print(" %12s   %8s               %8s                    %8s   Click\n" % ("CTR", "Cost (per click)", "Bidder ID","Expected Utility(Revenue)"))
for i,slot in enumerate(slotList):
    ClickTime=CalculateClick(i,slot.Owner,ClickRate,PositionCTR)
    print("slot %2.0f:  %5.4f    %8.2f      %s         %10.0f            %8.0f" % (i+1, float(slot.CTR), float(slot.Cost), slot.Owner,ClickTime*(TrueValue(slot.Owner,BidData)-slot.Cost),ClickTime))
ClickTime=0
# revenue=0
# for i in range(len(slots)):
#     revenue+=slots[i].Cost*slots[i].CTR*100*qua[slots[i].Owner]
# Welfare=0
# for i in range(len(slots)):
#     Welfare=Welfare+qua[int(slots[i].Owner)]*BIDDD[int(slots[i].Owner)]
# print()
# print("Social Welfare is: %.2f."%Welfare)
# print("Revenue is:       %.2f.          (100 views)"%revenue) 

          CTR   Cost (per click)               Bidder ID                    Expected Utility(Revenue)   Click

slot  1:  0.0186     4800.00      24fffcaf-134a-4a99-9756-f9261b74ed6a           56449626               47041
slot  2:  0.0102     3816.00      d42271d5-7367-4ffb-95a5-4d1cb2b0dc59          791621950              829792
slot  3:  0.0073     2000.00      6f15d700-facb-45be-aaba-bbd1fb359833          282319815              564640
slot  4:  0.0049     1000.00      8b105e0d-9c93-499f-b67f-2f2a26407871           49321450              197286
slot  5:  0.0043      744.00      722ba2cc-8354-4029-adf5-9b901b4afb5c         3026387400            16270900
slot  6:  0.0040      600.00      327b7d84-2646-4818-a1ee-a45a2e7d716c                  0                   0
slot  7:  0.0032      520.00      2de3966f-0109-4c7b-b4a1-fd1dde8b3bb4           15608053              120062
slot  8:  0.0028      432.00      409c8972-a5fa-4233-8b47-7d3d4e4ba961           11589444              107310
slot  9: 

# Start GSP Auction

In [166]:
auctionGSP = Auction(KeyWord,GSPBid)

In [167]:
auctionGSP.GSP(slotList)

In [168]:
SumB=0
SumE=0
SW=0
SumC=0
print(" %12s   %8s               %8s                    %8s         Click\n" % ("CTR", "Cost (per click)", "Bidder ID","Expected Utility"))
for i,slot in enumerate(slotList):
    ClickTime=CalculateClick(i,slot.Owner,ClickRate,PositionCTR)
    SumB+=ClickTime*(TrueValue(slot.Owner,BidData)-slot.Cost)
    SumE+=ClickTime*float(slot.Cost)
    SW+=ClickTime*TrueValue(slot.Owner,BidData)
    SumC+=ClickTime
    print("slot %2.0f:  %5.4f    %8.2f      %s         %10.0f            %8.0f" % (i+1, float(slot.CTR), float(slot.Cost), slot.Owner,ClickTime*(TrueValue(slot.Owner,BidData)-slot.Cost),ClickTime))
# revenue=0
# for i in range(len(slots)):
#     revenue+=slots[i].Cost*slots[i].CTR*100*qua[slots[i].Owner]
# Welfare=0
# for i in range(len(slots)):
#     Welfare=Welfare+qua[int(slots[i].Owner)]*BIDDD[int(slots[i].Owner)]
# print()
# print("Social Welfare is: %.2f."%Welfare)
# print("Revenue is:       %.2f.          (100 views)"%revenue) 

          CTR   Cost (per click)               Bidder ID                    Expected Utility         Click

slot  1:  0.0186     5605.49      ae30807c-5316-42ee-a031-589fdb08e4ba         4613887055              205569
slot  2:  0.0102     1224.56      6524d297-40d9-4a96-90c0-0d5aa0c61da6         1542848043              420917
slot  3:  0.0073      356.97      9eb5a22f-1870-42b1-aaae-2faa5235ae34          500613458              778518
slot  4:  0.0049      721.05      a2860a55-f8a1-4d62-8edc-49fb88661adc          175886238              137524
slot  5:  0.0043     1858.72      54bff083-a345-436d-99f4-3e70f8969567           95272044               50374
slot  6:  0.0040      761.85      ebd4f3cf-2d58-41d9-a848-e3db688ba491          300222767              242476
slot  7:  0.0032      640.97      b5f02143-b63e-4062-bb82-0624909e55ac          120150219              139868
slot  8:  0.0028      550.51      a980abdb-d307-4aa8-99f1-03e268964e0b            4031763               81466
slot  9:  0.

In [169]:
SumB

7374584389.809481

In [170]:
SumC

2484992.899123713

In [171]:
SumE

2532639737.089593

In [172]:
SW

9907224126.899073

# GSP no quality

In [173]:
auctionGSPn = Auction(KeyWord,GSPBid)
auctionGSPn.GSPnq(slotList)
SumB=0
SumE=0
SumC=0
print(" %12s   %8s               %8s                    %8s         Click\n" % ("CTR", "Cost (per click)", "Bidder ID","Expected Utility"))
for i,slot in enumerate(slotList):
    ClickTime=CalculateClick(i,slot.Owner,ClickRate,PositionCTR)
    SumB+=ClickTime*(TrueValue(slot.Owner,BidData)-slot.Cost)
    SumE+=ClickTime*float(slot.Cost)
    SumC+=ClickTime
    print("slot %2.0f:  %5.4f    %8.2f      %s         %10.0f            %8.0f" % (i+1, float(slot.CTR), float(slot.Cost), slot.Owner,ClickTime*(TrueValue(slot.Owner,BidData)-slot.Cost),ClickTime))

          CTR   Cost (per click)               Bidder ID                    Expected Utility         Click

slot  1:  0.0186     6024.65      ae30807c-5316-42ee-a031-589fdb08e4ba         4527719559              205569
slot  2:  0.0102     2784.49      52852966-92dd-43ac-8435-01b9e6021c7c                  0                   0
slot  3:  0.0073     1931.58      6524d297-40d9-4a96-90c0-0d5aa0c61da6         1127161931              381001
slot  4:  0.0049     1036.95      54bff083-a345-436d-99f4-3e70f8969567          158253787               58331
slot  5:  0.0043      900.76      a2860a55-f8a1-4d62-8edc-49fb88661adc          130552304              118766
slot  6:  0.0040      813.74      ebd4f3cf-2d58-41d9-a848-e3db688ba491          287639819              242476
slot  7:  0.0032      657.58      b5f02143-b63e-4062-bb82-0624909e55ac          117827495              139868
slot  8:  0.0028      610.79      9eb5a22f-1870-42b1-aaae-2faa5235ae34          112324667              288598
slot  9:  0.

In [174]:
SumC

1434607.63952944

In [175]:
SumB

6461479561.599486

In [176]:
SumE

2607436933.649024

In [177]:
SW=SumB+SumE
SW

9068916495.24851

# Start VCG Auction

In [178]:
auctionVCG = Auction(KeyWord,VCGBid)

In [179]:
auctionVCG.VCG(slotList)

In [180]:
SumB=0
SumE=0
SumC=0
print(" %12s   %8s               %8s                    %8s         Click\n" % ("CTR", "Cost (per click)", "Bidder ID","Expected Utility"))
for i,slot in enumerate(slotList):
    ClickTime=CalculateClick(i,slot.Owner,ClickRate,PositionCTR)
    SumB+=ClickTime*(TrueValue(slot.Owner,BidData)-slot.Cost)
    SumE+=ClickTime*float(slot.Cost)
    SumC+=ClickTime
    print("slot %2.0f:  %5.4f    %8.2f      %s         %10.0f            %8.0f" % (i+1, float(slot.CTR), float(slot.Cost), slot.Owner,ClickTime*(TrueValue(slot.Owner,BidData)-slot.Cost),ClickTime))
# revenue=0
# for i in range(len(slots)):
#     revenue+=slots[i].Cost*slots[i].CTR*100*qua[slots[i].Owner]
# Welfare=0
# for i in range(len(slots)):
#     Welfare=Welfare+qua[int(slots[i].Owner)]*BIDDD[int(slots[i].Owner)]
# print()
# print("Social Welfare is: %.2f."%Welfare)
# print("Revenue is:       %.2f.          (100 views)"%revenue) 

          CTR   Cost (per click)               Bidder ID                    Expected Utility         Click

slot  1:  0.0186     6024.65      ae30807c-5316-42ee-a031-589fdb08e4ba         4527719559              205569
slot  2:  0.0102     2784.49      52852966-92dd-43ac-8435-01b9e6021c7c                  0                   0
slot  3:  0.0073     1931.58      6524d297-40d9-4a96-90c0-0d5aa0c61da6         1127161931              381001
slot  4:  0.0049     1036.95      54bff083-a345-436d-99f4-3e70f8969567          158253787               58331
slot  5:  0.0043      900.76      a2860a55-f8a1-4d62-8edc-49fb88661adc          130552304              118766
slot  6:  0.0040      813.74      ebd4f3cf-2d58-41d9-a848-e3db688ba491          287639819              242476
slot  7:  0.0032      657.58      b5f02143-b63e-4062-bb82-0624909e55ac          117827495              139868
slot  8:  0.0028      610.79      9eb5a22f-1870-42b1-aaae-2faa5235ae34          112324667              288598
slot  9:  0.

In [181]:
SumB

6461479561.599487

In [182]:
SumE

2607436933.649024

In [183]:
SW=SumB+SumE
SW

9068916495.248512

In [184]:
SumC

1434607.63952944